<a href="https://colab.research.google.com/github/PavelStelmakhV/hw309-selection-of-hyperparameters/blob/main/hw309.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import layers
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical


# import numpy as np
# import matplotlib.pyplot as plt
# import tf_keras

# from keras import layers
# from keras import models
# from keras import regularizers
# from keras import callbacks
# from keras import initializers

# from keras.layers import BatchNormalization
# from keras.models import load_model


In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
w_init_tanh = initializers.glorot_normal(seed=111)
w_init_relu = initializers.HeNormal(seed=66)
# w_init_relu = initializers.HeUniform(seed=24)
b_init = initializers.Zeros()